# 0. 들어가며
 
## 0.1 추천시스템이 뭐였지

1. 내용 기반 필터링
* 콘텐츠의 유사도를 기반으로 사용자에게 추천
2. 협업 필터링
* 서비스 안의 다양한 사용자로부터 얻은 선호도 정보 기반으로 사용자에게 추천
3. 하이브리드 필터링
* 2가지 이상의 다양한 종류의 추천시스템 알고리즘 조합해서 만들어진 알고리즘

* 협업 필터링의 구현 방식
    1. 기억 기반
    * 실제 평점 데이터 메모리에 저장하고 유사도 계산
        1. 사용자 기반 협업 : 나와 비슷한 취향의 사용자
        2. 아이템 기반 협업 : 함께 선호되는 아이템
    * 문제점
        1. 데이터 희소성 문제
        2. 확장성 문제
        3. 새로운 아이템 문제
    * 따라서 이를 극복하고자 모델 기반 협업 필터링이 등장하였다!

# 1. 인트로 - 모델 기반

## 1.1 잠재 요인과 행렬 분해

### 1.1.1 잠재 요인
* 사용자-아이템 상호작용 행렬에 숨어있는 사용자의 취향 및 아이템의 특성을 설명하는 숨겨진 차원
    * 사람이 보고 알 수는 없고, 모델이 스스로 학습하는 추상적인 개념
### 1.1.2 행렬 분해
*  평점 행렬이 취향 벡터와 아이템 특성 벡터의 곱이라고 생각하고, 이를 가장 잘 재현하는 가장 적절한 잠재 요인의 개수 k를 찾아 평점 행렬을 나눠내는것
### 1.1.3 MF 수행 방식
1. SVD
    * 어떤 직사각형 행렬이든 3 행렬로 분리해내는 방법
    * 직교행렬 2개와 특잇값으로 분해가됨
    * 직교행렬 2개로 차원축소
    * 실제 PF추천 시스템의 희소한 평점 행렬에 적용하기에는 한계가 존재 : 희소성 처리 불가, 계산 복잡성
2. SGD 
    * 오차의 제곱합을 최소화하고 오버피팅 방지를 위한 정규화 진행
    * 경사 하강법을 통해 오차를 줄이는 방향으로 업데이트
3. NMF
    * 행렬 분해에서 분해된 행렬의 모든 원소가 0이상이라는 제약 조건을 추가
    * 직관적 의미 부여 가능!
4. PMF
    * 누가 무엇을 좋아할지 예측하는 데에 목표를 둔다
    * 확률적 모델로 접근
    * 불확실성을 포함!

# 2 . 최근 추천시스템의 방향성

## 2.1 VAE 기반 협업 필터링
* 사용자의 아이템 상호작용 벡터를 입력받아 잠재 공간에서 재구성하는 VAE 학습
* 사용자 이력을 z로 매핑, z로부터 아이템 분포 복원
* 장점 : 확률적 잠재 분포를 통해 불확실성을 반영

### 2.1.1 MF의 한계와 딥러닝
* 한계 : 선형성의 한계와 불확실성 무시

### 2.1.2 왜 VAE?
* AE는 데이터가 부족하면 틀리기 쉬워 노이즈에 취약하다.
* VAE는 분포로 받아들여 취향의 불확실성을 높인다

### 2.1.3 VAE기반 CF의 작동구조
* 사용자의 아이템 상호작용 벡터를 입력으로 받아 잠재 공간에서 이를 재구성하도록 하는 학습 방식

### 2.1.4 VAE의 비밀병기 : 목적 함수와 KL Divergence
* VAE의 우수성은 목적 함수 담당, 두가지 목표를 동시에 최적화
1. 재구성 오차 최적화 : 디코더가 원래 입력과 최대한 비슷하게 복원하도록 만듬
2. KL Divergence 항 : 인코더 출력 잠재 분포가 사전 분포와 유사하도록 강제
    * 잠재 공간을 정규 분포 근처로 압축해서 부드럽게 만듬
    * 따라서 값들 사이의 보간이 가능하다
3. 장점
    * 불확실성 반영 가능 
    * 희소한 데이터 상호작용에서도 강력
    * 비선형성 학습

## 2.2 GNN 기반 사용자 선호 예측
### 2.2.0 GNN의 필요성
*  VAE도 관계 및 순서와 같은 풍부한 정보는 활용하지는 못한다. 상호작용을 개별 집합으로 이용하기 때문
* 그렇다면 관계 자체를 데이터로 볼 수는 없을까?
### 2.2.1 GNN
* GNN은 사용자와 아이템이 형성하는 거대한 연결망에 집중
* 그래프 : 노드와 그 노드를 잇는 선 엣지를 모아 구성한 자료 구조
    * 노드는 사용자, 엣지는 사용자 간 연결
    * 쓰는 이유?
        1. 관계, 상호작용과 같이 추상적인 개념을 다루기에 좋음
        2. 유클리디안 공간이 아니어도 적용가능

### 2.2.2 GNN의 구조
* 핵심 아이디어 : 내 이웃 노드의 정보를 나에게 전달/요약
1. 그래프 구조 생성
    * 노드와 엣지를 정의하고 각각의 피처를 담아 표현
    * 정보와 정보 사이의 상호작용인 엣지가 형성
2. 메시지 패싱
    * 그래프에서 노드 간 정보를 전달하고 상호작용하는 방법
    1. 타겟 노드 설정
    2. 메시지 생성 : 각 노드가 이웃에게 보내는 자신의 정보
    3. 메시지 집계 : 여러 이웃 노드에서 받은 메시지를 하나의 벡터로 통합
    4. 갱신 : 이웃 정보와 자신의 이전 상태를 결합해 새로운 임베딩을 계산
* 결과적으로, 이웃들의 취향까지 반영된 최종 임베딩을 얻을 수 있음.

### 2.2.3 LightGCN
* 다양한 아키텍쳐들
    * GCN : 그래프 위에서 convolution 계산을 시행
    * GRN : RNN + GNN -> 시간에 따라 노드 특성이 업데이트, 그래프의 동적 변화포착
    * GAT : 이웃 노드 중요도 학습해 가중 평균
    * GraphSAGE : 대규모 그래프에 대해 샘플링 기반 학습

* LightGCN : GNN에서 구성 요소 2가지 제거
    * 특징 변환 제거
    * 비선형 활성화 함수 제거
    * 이에 따라 정규화된 이웃 임베딩의 합으로만 구성, 가볍고 효율적

### 2.2.4 GNN 활용 사례
* 핀터레스트 : 사용자들이 상호작용하는 핀, 보드, 카테고리를 분석해 추천
* SNS : 친구 추천 등
* 화학, 생명과학 
* 금융 및 보안
* 프로그램 분석

### 2.2.25 GNN의 장단점
* 장점
    1. 고차원 협업 시그널 : 이웃 데이터를 반복 반영가능
    2. 정형화되지 않은 데이터 학습 기능 : 관계만 있으면 뭐든 가능
    3. Cold start 문제 완화 효과적 : 근처를 이용한 즉각 임베딩

* 단점
    1. 과평탄화 : 모든 노드의 임베딩이 같은 값으로
    2. 과압축 : 이웃이 많은데 하나의 벡터로 하려니 문제
    3. 계산 복잡도 업
    4. 해석의 어려움 : 순서가 없음

# 2.3 Transformer 기반 추천 시스템
* 순서와 문맥으로 사용자 이해

* 기존 추천 시스템의 순차적 한계 
1. CF의 순차적 한계
    * 순서 무시 : 내용만 같으면 같다
    * 문맥 부족 : 언제라는게 없음
    * 예측의 한계 : 다음 행동 예측보다 일반적인 선호도 찾는 데에 초점
2. GNN의 순차적 한계
    * 지역적 정보 전파의 한계 : 여러 단계를 거치면서 희석 가능
    * 장기 의존성 문제

* 사용자 행동을 순서로 접근하는 트랜스포머 : 문맥의 파악이 가능
    1. 셀프 어텐션
        * 다른 아이템과 함꼐 결합되어 문맥에 따라 내용이 결정
    2. long range dependency
        * 행동이 길어질 때 앞부분의 행동을 잊어버리지 않고 멀리 떨어진 과거 행동이 다음 행동의 결정적 키가 된다.
    3. self - attention 매커니점
        * QKV를 이용해서 가중치를 관련성에 맞게 부가해준다..!!

* 트랜스포머 기반 주요 추천 시스템 모델
1. SASRec
    * 단방향 어텐션
    * 현재 아이템은 과거의 아이템에 의존
    * 이를 기반으로 다음 아이템을 예측
    * 특징 : 시간의 흐름을 명확하게 반영하며
    * 문제 : 계산이 많고 미래를 참조하지 못함
2. BERTRec
    * 양방향 어텐션
    * 시퀀스 일부를 마스킹하고 과거 미래 모든 아이템을 참조해서 에측
    * 양방향 문맥 모두 고려
    * 시퀀스 전체의 선호도 파악에 초점

# 3. 진짜 최신 트렌드 : LLM의 시대
* 관점 전환 : 추천을 랭킹으로 매기는 것이 아니라 언어로 풀어낸다, 즉 이유에 대한 서술이 가능해진것!
* 새로운 가능성
    * 대화하듯이 질문을 받고 추천이 가능
    * 제로샷 추천 : 학습되지 않아도 텍스트 설명만으로도 추천이 가능 -> 콜드 스타트를 해결!

* P5 : 모든 추천 과제를 LLM이 풀어내도록 하나의 거대한 텍스트 프롬프트로 통일
* TALLRec : P5 비용을 절감하기 위해 이미 만들어진 LLM을 fine tuning해서 쓰자


# 4. 현업의 고민 : 책임감 있는 AI
* 모델 성능 외에도 신경써야되는게...?
1. 인기 편향
    * 소수의 인기 템에만 집중하고 다수의 비인기 템은 관심을 안 주는 현상
    * 살짝 롱테일 느낌
2. 설명가능성
    * 이걸 왜 추천했는지 이유 제시가 되는가?
    * 블랙박스의 문제!

* -> 즉, 정확성 뿐만이 아니라 논리까지 댈 수 있는 책임감이 필요
* 그럼 딥러닝 쓰면 되잖나?
* 문제 : 너무 비쌈
    * GPU를 써야하고
    * 서버비도 내야하고
    * 이걸 유지까지 해야함
* 따라서, 선형 기반 추천 시스템과 병행!
    1. 후보군 생성
        * 매우 많은 템에서 사용자가 조금이라도 흥미있을 템을 선형 모델로 추려냄
    2. 순위 매기기
        * 추려진 템을 딥러닝 모델로 최고 정확도를 동반해서 정교한 순서를 매긴다